### Project

In [0]:
# Import section
import torch as torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np


##### Defining values of batch size and defining transformations

In [0]:
batch_size = 64
test_batch_size = 64
mnist_batch_size = 64
train_batch_size=64
torch.manual_seed(7)
np.random.seed(7)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Transformations
data_transformations = transforms.Compose([
    transforms.ToTensor(),

    transforms.Normalize((0.1307,), (0.3081,))
])
test_transformations = transforms.Compose([

    transforms.Grayscale(3),
    transforms.transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])


##### Load Data, create data loaders

In [0]:
# Data Source
mnist_train = datasets.MNIST('../data', train=True, download=True,
                                 transform=test_transformations)
mnist_test = datasets.MNIST('../data', train=False, download=True,
                                transform=test_transformations)
svhn_train = datasets.SVHN('../data', split='train', download=True,
                               transform=data_transformations)
svhn_test = datasets.SVHN('../data', split='test', download=True,
                              transform=data_transformations)

# Data loaders
train_loader = DataLoader(svhn_train,
                          batch_size=batch_size, shuffle=True)

mnist_test_loader = DataLoader(mnist_test,
                          batch_size=mnist_batch_size, shuffle=True)
svhn_test_loader = DataLoader(svhn_test,
                          batch_size=mnist_batch_size, shuffle=True)
svhn_train_loader = DataLoader(svhn_train,
                          batch_size=mnist_batch_size, shuffle=True)

mnist_train_loader = DataLoader(mnist_train,
                          batch_size=mnist_batch_size, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 20355842.93it/s]                            


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


32768it [00:00, 312100.23it/s]                           
0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


...

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw
Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...


  0%|          | 0/182040794 [00:00<?, ?it/s]

Done!


182042624it [00:06, 29625166.90it/s]                               
  0%|          | 16384/64275384 [00:00<08:30, 125804.78it/s]

64282624it [00:04, 14694639.06it/s]                              


###### Xavier weight initialization

In [0]:
def init_weights(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
      torch.nn.init.xavier_uniform(m.weight)

#### Model definining section

###### Feature Extractor

In [0]:
class FeatureExtractor(nn.Module):
  def __init__(self):
    super(FeatureExtractor, self).__init__()
    self.layer1 = nn.Sequential(
                        nn.Conv2d(3,64,5,padding=1),   # batch x 16 x 28 x 28
                        nn.ReLU(),
                        nn.MaxPool2d(3, stride=2),
                        nn.Conv2d(64,64,5,padding=1),
                        nn.ReLU(),
                        nn.MaxPool2d(3, stride=2),
                        nn.Conv2d(64,128,5,padding=1),
                        nn.ReLU(),
                        nn.MaxPool2d(3, stride=2),
                        )
    self.layer1.apply(init_weights)
  def forward(self, x):
    out = self.layer1(x)
    return out.view(x.size()[0], -1)


###### Defining Gradient Reversal Layer

In [0]:
class GradRevLayer(torch.autograd.Function):
  lambd = 1.0
  @staticmethod
  def forward(ctx,x):
    return x.view_as(x)
  
  @staticmethod
  def backward(ctx, out_grads):
    return GradRevLayer.lambd * out_grads.neg()
  

def grad_reverse_layer(x, new_lambd):
    GradRevLayer.lambd = new_lambd
    return GradRevLayer.apply(x)

##### Domain Classifier

In [0]:
class DomainClassifier(nn.Module):
  def __init__(self):
    super(DomainClassifier, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Linear(128, 1024),
        nn.ReLU(),
        nn.Linear(1024, 1024),
        nn.ReLU(),
        nn.Linear(1024, 2),
        nn.LogSigmoid()
    )
    self.layer1.apply(init_weights)
  def forward(self, x):
    x = grad_reverse_layer(x,self.lambd)
    return self.layer1(x)
  def set_lambda(self, lambd):
    self.lambd = lambd


##### LabelPredictor


In [0]:
class LabelPredictor(nn.Module):
  def __init__(self):
    super(LabelPredictor, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Linear(128, 3072),
        nn.ReLU(),
        nn.Linear(3072, 2048),
        nn.ReLU(),
        nn.Linear(2048, 10),
        nn.LogSoftmax()
    )
    self.layer1.apply(init_weights)
  def forward(self, x):
    return self.layer1(x)

##### Create dataset for domainclasiifier



In [0]:
X_mnist = []
for i in range(len(mnist_train)):
  X_mnist.append(mnist_train.__getitem__(i)[0])


In [0]:
X_mnist_tensor = torch.stack(X_mnist)
y_mnist = torch.zeros(len(X_mnist))
X_svhn = torch.tensor(svhn_train.data[:len(X_mnist)])
y_svhn = torch.ones(len(X_svhn))
X = torch.cat((X_mnist_tensor.float(),X_svhn.float()))
y = torch.cat((y_mnist,y_svhn))
#img = X_mnist.__getitem__(0)
#img.shape



In [0]:
X.shape

torch.Size([120000, 3, 32, 32])

In [0]:
ds_dataset = torch.utils.data.TensorDataset(X,y)

In [0]:
classifier_loader = DataLoader(ds_dataset,
                          batch_size=train_batch_size, shuffle=True)

#### Training


##### Initializing Models


In [0]:
feature_extractor = FeatureExtractor().cuda()
label_predictor = LabelPredictor().cuda()
domain_classifier = DomainClassifier().cuda()

parameters = [param for param in feature_extractor.parameters()]
parameters_domain = [param for param in domain_classifier.parameters()]
parameters_classifier = [param for param in label_predictor.parameters()]
parameters += (parameters_domain)
parameters += (parameters_classifier)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
# Testing model
def test(feature_extractor, class_predictor, device, test_loader):
    feature_extractor.eval()
    class_predictor.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            # print(data.shape)
            output = class_predictor(feature_extractor(data))
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
       test_loss, correct, len(test_loader.dataset),
       100. * correct / len(test_loader.dataset)))
    return 100. * correct / len(test_loader.dataset)

###### Training part


In [0]:

# Usage of CUDA
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

#Parameters
epoch_num = 200
log_interval = 200
lambd = 1
gamma = 10

svhn_test_accuracies = []
mnist_test_accuracies = []
svhn_train_accuracies = []
total_steps = epoch_num * len(train_loader)

optimizer = optim.Adam(parameters)
new_optimizer = optim.SGD(parameters,lr=0.01, momentum=0.9)
for epoch in range(epoch_num):
  start_steps = epoch * len(train_loader)
  dataloader_iterator = iter(enumerate(mnist_train_loader))
  print("Epoch num is ", epoch)
  feature_extractor.train()
  label_predictor.train()
  domain_classifier.train()
  for batch_idx, (data, target) in enumerate(train_loader):
      p = float(batch_idx + start_steps) / total_steps
      new_lambda = 2. / (1. + np.exp(- gamma * p)) - 1
      domain_classifier.set_lambda(new_lambda)
      data, target = data.to(device), target.to(device)
      for param_group in new_optimizer.param_groups:
       param_group['lr'] = 0.01 / (1. + 10 * p) ** 0.75
      new_optimizer.zero_grad()
      # print(data.shape)
      y_svhn = torch.ones(len(data))
      
      features = feature_extractor(data)
      features = features.to(device)
      output = label_predictor(features)
      output_source = domain_classifier(features)
      #print(output) 
      #print(output.shape)
      #print(target.shape)
      loss = F.nll_loss(output, target)
      loss.backward(retain_graph=True)
      #predictor_optimizer.step()
      if batch_idx % log_interval == 0:
          print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
              epoch, batch_idx * len(data), len(train_loader.dataset),
                  100. * batch_idx / len(train_loader), loss.item()))
      try:
        d_batch_idx, (d_data, d_target) = next(dataloader_iterator)
      except StopIteration:
            dataloader_iterator = iter(enumerate(mnist_train_loader))
            d_batch_idx, (d_data, d_target) = next(dataloader_iterator)
      data, target = d_data.to(device), d_target.to(device)
      y_mnist = torch.zeros(len(data))
      #classifier_optimizer.zero_grad()
      # print(data.shape)
      features = feature_extractor(data)
      features = features.to(device)
      output = domain_classifier(features) 
      # print(output.shape)
      #print(output)
      #print(target)
      criterion = nn.CrossEntropyLoss()
      loss = criterion(output, y_mnist.long().to(device)) * lambd + criterion(output_source, y_svhn.long().to(device)) * lambd
      loss.backward()
      new_optimizer.step()
      if batch_idx % log_interval == 0:
          print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
              epoch, batch_idx * len(data), len(mnist_train_loader.dataset),
                  100. * batch_idx / len(mnist_train_loader), loss.item()))
  print("Accuracy on SVHN test:")
  svhn_test_accuracies.append(test(feature_extractor, label_predictor, device, svhn_test_loader))
  print("Accuracy on MNIST test:")
  mnist_test_accuracies.append(test(feature_extractor, label_predictor, device, mnist_test_loader))
  print("Accuracy on SVHN train:")
  svhn_train_accuracies.append(test(feature_extractor, label_predictor, device, svhn_train_loader))




Epoch num is  0
Train Epoch: 0 [0/73257 (0%)]	Loss: 2.310196
Train Epoch: 0 [0/60000 (0%)]	Loss: 1.389425


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Train Epoch: 0 [12800/73257 (17%)]	Loss: 1.882573
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.003443
Train Epoch: 0 [25600/73257 (35%)]	Loss: 0.784596
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.719676
Train Epoch: 0 [38400/73257 (52%)]	Loss: 0.493824
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.711382
Train Epoch: 0 [51200/73257 (70%)]	Loss: 0.467576
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.768076
Train Epoch: 0 [64000/73257 (87%)]	Loss: 0.586269
Train Epoch: 0 [64000/60000 (107%)]	Loss: 0.713442
Accuracy on SVHN test:

Test set: Average loss: 0.4287, Accuracy: 22716/26032 (87.26%)

Accuracy on MNIST test:

Test set: Average loss: 3.3318, Accuracy: 5915/10000 (59.15%)

Accuracy on SVHN train:

Test set: Average loss: 0.3774, Accuracy: 65029/73257 (88.77%)

Epoch num is  1
Train Epoch: 1 [0/73257 (0%)]	Loss: 0.377133
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.724585
Train Epoch: 1 [12800/73257 (17%)]	Loss: 0.494736
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.733836
Train Epoch: 1 [25600/73257 